# Siamese neural network
first we check what is the type of the x and y in the example i got from https://www.youtube.com/watch?v=DGJyh5dK4hU

In [1]:
# import tensorflow as tf

# mnist = tf.keras.datasets.mnist

# (X_train, y_train), (X_test, y_test) = mnist.load_data()

# print(type(y_train))
# X_train[0][0]

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping

import itertools

In [2]:
img = cv2.imread('actor_faces\Aaron_Eckhart\Aaron_Eckhart_1_1.jpeg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(len(img[0]))

365


In [4]:
data_dir = "actor_faces"
face_data = []
label_data = []

for label in os.listdir(data_dir):
    label_path = os.path.join(data_dir, label)
    
    if not os.path.isdir(label_path):
        continue
    
    for file_name in os.listdir(label_path):
        cur_path = os.path.join(label_path, file_name)
        cur_img = cv2.imread(cur_path)
        if cur_img is None:
          continue
        cur_img = cv2.resize(cur_img, (100, 100))
        cur_img = cv2.cvtColor(cur_img, cv2.COLOR_BGR2GRAY)
        face_data.append(cur_img)
        label_data.append(label)

print(label_data[0]) 
print(face_data[0])

cv2.imshow('anjay', face_data[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

Aaron_Eckhart
[[118  95 114 ... 135 133 146]
 [106 102 123 ... 134 130 137]
 [ 97 111 133 ... 134 130 134]
 ...
 [  6   7   7 ...  62  60  57]
 [  6   6   9 ...  56  51  51]
 [  6   5  11 ...  55  53  61]]


In [5]:
face_array = np.array(face_data)
label_array = np.array(label_data)

print(face_array.shape)
print(label_array.shape)

(13233, 250, 250)
(13233,)


In [6]:
x_train, x_test, y_train, y_test = train_test_split(face_array, label_array, test_size=0.2, random_state=42)

# Check the shapes of the split arrays
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (10586, 250, 250)
y_train shape: (10586,)
x_test shape: (2647, 250, 250)
y_test shape: (2647,)


In [7]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model


img_A_inp = Input((250, 250), name='img_A_inp')
img_B_inp = Input((250, 250), name='img_B_inp')

def get_cnn_block(depth):
  return Sequential([Conv2D(depth, 3, 1),
                     BatchNormalization(),
                     ReLU()])

DEPTH = 64
cnn = Sequential([Reshape((250, 250, 1)),
                  get_cnn_block(DEPTH),
                  get_cnn_block(DEPTH*2),
                  get_cnn_block(DEPTH*4),
                  get_cnn_block(DEPTH*8),
                  GlobalAveragePooling2D(),
                  Dense(64, activation='relu')])

feature_vector_A = cnn(img_A_inp)
feature_vector_B = cnn(img_B_inp)

concat = Concatenate()([feature_vector_A, feature_vector_B])

dense = Dense(64, activation='relu')(concat)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[img_A_inp, img_B_inp], outputs=output)

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ img_A_inp           │ (None, 250, 250)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ img_B_inp           │ (None, 250, 250)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_4        │ (None, 64)        │  1,586,496 │ img_A_inp[0][0],  │
│ (Sequential)        │                   │            │ img_B_inp[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128)       │          0 │ sequential_4[0][… │
│ (Concatenate)       │                   │            │ sequential_4[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,594,817 (6.08 MB)

 Trainable params: 1,592,897 (6.08 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [8]:
random_indices = np.random.choice(x_train.shape[0], 300, replace=False)

X_train_sample, y_train_sample = x_train[random_indices], y_train[random_indices]

X_train_sample.shape, y_train_sample.shape

((300, 250, 250), (300,))

In [9]:
len(X_train_sample) ** 2

90000

In [10]:


def make_paired_dataset(X, y):
  X_pairs, y_pairs = [], []
import itertools
  tuples = [(x1, y1) for x1, y1 in zip(X, y)]
  
  for t in itertools.product(tuples, tuples):
    pair_A, pair_B = t
    img_A, label_A = t[0]
    img_B, label_B = t[1]

    new_label = int(label_A == label_B)

    X_pairs.append([img_A, img_B])
    y_pairs.append(new_label)
  
  X_pairs = np.array(X_pairs)
  y_pairs = np.array(y_pairs)

  return X_pairs, y_pairs

In [11]:
make_paired_dataset(X_train_sample, y_train_sample)

(array([[[[  3,   3,   4, ...,   0,   0,   0],
          [  2,   2,   2, ...,   0,   0,   0],
          [  1,   1,   1, ...,   0,   0,   0],
          ...,
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0]],
 
         [[  3,   3,   4, ...,   0,   0,   0],
          [  2,   2,   2, ...,   0,   0,   0],
          [  1,   1,   1, ...,   0,   0,   0],
          ...,
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0]]],
 
 
        [[[  3,   3,   4, ...,   0,   0,   0],
          [  2,   2,   2, ...,   0,   0,   0],
          [  1,   1,   1, ...,   0,   0,   0],
          ...,
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0]],
 
         [[  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0],
   

In [12]:
X_train_pairs, y_train_pairs = make_paired_dataset(X_train_sample, y_train_sample)

X_train_pairs.shape, y_train_pairs.shape

((90000, 2, 250, 250), (90000,))

In [13]:
random_indices = np.random.choice(x_test.shape[0], 150, replace=False)

X_test_sample, y_test_sample = x_test[random_indices], y_test[random_indices]

X_test_sample.shape, y_test_sample.shape

((150, 250, 250), (150,))

In [14]:
X_test_pairs, y_test_pairs = make_paired_dataset(X_test_sample, y_test_sample)

X_test_pairs.shape, y_test_pairs.shape

((22500, 2, 250, 250), (22500,))

In [15]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3)

: 

In [ ]:
model.fit(x=[X_train_pairs[:, 0, :, :], X_train_pairs[:, 1, :, :]],
          y=y_train_pairs,
          validation_data=([X_test_pairs[:, 0, :, :], 
                            X_test_pairs[:, 1, :, :]],
                           y_test_pairs),
          epochs=50,
          batch_size=16,
          callbacks=[es])

Epoch 1/50


c:\Users\Deciano\.virtualenvs\siamese_face_recog-2vr_2qQt\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['img_A_inp', 'img_B_inp']. Received: the structure of inputs=('*', '*')
  warnings.warn(


In [ ]:
img_A, img_B = X_test[0], X_test[17]
label_A, label_B = y_test[0], y_test[17]

label_A, label_B